In [5]:
# import necessary stuff and python-wrapper of verse
import os
import numpy as np
from scipy.sparse import csr_matrix

from verse.python.wrapper import VERSE

In [6]:
# define path to c++ inplementation of verse and its *.so compiled files
# instantiate verse algorithm object
cpath = os.path.dirname(os.path.realpath('verse/src/verse.cpp'))
verse = VERSE(cpath=cpath)

In [7]:
# construct example adjacency matrix in csr format
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])

csr_adjacency_matrix = csr_matrix((data, (row, col)), shape=(3, 3))

In [8]:
# define hyper-parameters for verse with personalized page rank as sim_G function
w=None
n_hidden=128
steps=100000
n_neg_samples=3
lr=0.0025
rng_seed=0
n_threads=-1
graph = csr_adjacency_matrix

In [9]:
# compute node embeddings
# ATTENTION: takes a while until termination and cannot be terminated through restarting kernel
# Need to kill terminal, where jupyter notebook is started in in order to kill c++-process
verse.verse_ppr(graph)

array([[  3.09714638e-02,  -2.41056830e-02,   1.92041579e-03,
         -4.46759947e-02,   7.33025745e-02,  -8.99075270e-02,
          2.01863870e-02,  -6.20665736e-02,   3.10214981e-03,
         -3.63472886e-02,   3.77487876e-02,  -5.90696279e-03,
         -5.69981150e-02,   2.33960040e-02,   8.00003484e-02,
          5.90029284e-02,  -5.68658626e-03,  -4.60878909e-02,
         -8.96013007e-02,   1.03367995e-02,   3.79473553e-03,
         -3.28600928e-02,   1.28807710e-03,  -2.11275630e-02,
         -1.34934373e-02,  -6.65321900e-03,  -3.60931009e-02,
         -9.45151597e-02,   7.26752058e-02,   8.22316632e-02,
         -2.20550373e-02,  -9.60748084e-03,  -1.14849359e-02,
          1.77333187e-02,   5.30914366e-02,   6.60800934e-02,
          5.76567948e-02,   3.47503275e-02,   4.28789668e-03,
          1.61261279e-02,  -2.35955138e-02,   2.96037663e-02,
         -3.55260037e-02,  -2.59315707e-02,   2.15825718e-02,
          3.29316817e-02,   1.13760389e-01,   7.56176412e-02,
        

In [10]:
# get dimensionality of matrix described by csr format
graph.shape

(3, 3)